In [151]:
import torch
import os

nums = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '.']
symbols = set()
with open('input.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        for char in line.strip():
            if char not in nums:
                symbols.add(char)


def read_data(file_path: str) -> torch.Tensor:
    """
    Read data from a file and store it in a 140x140 tensor.
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Initialize a 140x140 tensor to store characters
    tensor = torch.zeros((140, 140), dtype=torch.int8)
    list_of_stars = []
    for i, line in enumerate(lines):
        for j, char in enumerate(line.strip()):
            tensor[i][j] = ord(char)
            if char == '*':
                list_of_stars.append((i, j))
                #print(f"star position: {i}, {j}")

    return tensor, list_of_stars

def get_adjacent_cells(i: int, j: int, length: int, rows: int, cols: int) -> set:
    """
    Get all adjacent cells for a number of given length, including diagonals.
    """
    adjacent_cells = set()

    # Cells before the number
    if j > 0:
        for x in range(i - 1, i + 2):
            if 0 <= x < rows:
                adjacent_cells.add((x, j - 1))
        # Add diagonal cells at the beginning of the line
        if i > 0:
            adjacent_cells.add((i - 1, j - 1))
        if i < rows - 1:
            adjacent_cells.add((i + 1, j - 1))

    # Cells after the number
    if j + length < cols:
        for x in range(i - 1, i + 2):
            if 0 <= x < rows:
                adjacent_cells.add((x, j + length))
        # Add diagonal cells at the end of the line
        if i > 0:
            adjacent_cells.add((i - 1, j + length))
        if i < rows - 1:
            adjacent_cells.add((i + 1, j + length))

    # Cells above and below the number
    for offset in range(length):
        if i > 0:
            adjacent_cells.add((i - 1, j + offset))
        if i < rows - 1:
            adjacent_cells.add((i + 1, j + offset))

    return adjacent_cells

def find_valid_numbers(tensor: torch.Tensor) -> set:
    """
    Find multi-digit numbers that have at least one adjacent cell with a character other than '.'.
    """
    rows, cols = tensor.shape
    valid_numbers = []
    not_valid = []
    any_number = {}
    index = 0
    for i in range(rows):
        j = 0
        while j < cols:
            if chr(tensor[i, j].item()).isdigit():
                start_j = j
                # Find the end of the number
                while j < cols and chr(tensor[i, j].item()).isdigit():
                    j += 1
                num_len = j - start_j
                number = int(''.join(chr(tensor[i, k].item()) for k in range(start_j, j)))
                index += 1
                this_number = (number, i, start_j, num_len)
                #print(f"this number =  {number} with the coordinates and length {this_number}")
                any_number[index] = this_number 
                # Get adjacent cells
                adjacent_cells = get_adjacent_cells(i, start_j, num_len, rows, cols)
                o = 1
                for x, y in adjacent_cells:
                    charac = chr(tensor[x, y].item())
                    if charac in symbols:
                        valid_numbers.append(number)
                        o=1
                    else:
                        o = 0
                if o == 0:
                    # no adjacent symbol
                    not_valid.append(number)
            else:
                j += 1

    return valid_numbers, not_valid, any_number

# Read data from file
file_path = 'input.txt'
tensor, list_of_stars = read_data(file_path)

# Find valid numbers
valid_numbers, not_valid, any_number = find_valid_numbers(tensor)
list_of_matches = []
for star_num, star in enumerate(list_of_stars):
    #print(star_num)
    matches = {}
    #print(f"Working on star {star_num} at position {star}")
    adjacent_cells = get_adjacent_cells(star[0], star[1], 1 , 140, 140)
    #print(f"Checking these adjacent cells: {adjacent_cells}")
    #print(star_num, star, adjacent_cells)
    for number in any_number.keys():
            for cell in adjacent_cells:
                x, y = cell[0], cell[1]
                l = len(str(any_number[number][0]))
                #print(l)
                for xx in range(l):
                    h, j = any_number[number][1], any_number[number][2] + xx
                    if x == h and y == j:
                        if star_num in matches.keys():
                            matches[star_num].append(any_number[number][0])
                        else:
                            matches[star_num] = [any_number[number][0]]
                            
            
    list_of_matches.append(matches)    
    



In [161]:
print('\n'.join(str(item) for item in list_of_matches[:20]))

# unique_data = [{key: list(set(value))} for d in list_of_matches for key, value in d.items()]
# print(unique_data[:20], "\n")

{0: [65]}
{1: [998, 998, 874]}
{2: [453, 453, 453, 407]}
{3: [845, 845, 845, 240, 240]}
{4: [773, 773, 773, 345]}
{5: [105, 105, 105]}
{6: [307, 298]}
{7: [527, 164]}
{8: [125, 593]}
{9: [331, 516]}
{10: [76, 196]}
{11: [861, 861, 377, 377, 377]}
{12: [752, 70]}
{13: [175, 839]}
{14: [74, 74, 899, 899]}
{15: [289, 418]}
{16: [750, 750, 468, 468]}
{17: [497, 366]}
{18: [258]}
{19: [745, 745, 745, 234]}


In [162]:

# Using dictionary comprehension to remove duplicates
unique_data = [{key: list(set(value))} for d in list_of_matches for key, value in d.items()]

#print(unique_data)
print('\n'.join(str(item) for item in unique_data[:20]))


{0: [65]}
{1: [874, 998]}
{2: [453, 407]}
{3: [240, 845]}
{4: [345, 773]}
{5: [105]}
{6: [298, 307]}
{7: [164, 527]}
{8: [593, 125]}
{9: [331, 516]}
{10: [196, 76]}
{11: [377, 861]}
{12: [752, 70]}
{13: [839, 175]}
{14: [74, 899]}
{15: [289, 418]}
{16: [468, 750]}
{17: [497, 366]}
{18: [258]}
{19: [745, 234]}


In [146]:
final_list = []
for match in list_of_matches:
    
    for key, val in match.items():
        set_of_numbers = set()
        for v in val:
            set_of_numbers.add(v)
        if len(set_of_numbers) == 2:
            final_list.append(set_of_numbers)
            

final_list
answer = 0
for x, y in final_list:
    #print(x * y)
    answer += (x * y)
print(answer)

80253814


In [48]:
any_number

{65: (0, 13, 2),
 998: (93, 135, 3),
 453: (82, 100, 3),
 845: (22, 122, 3),
 773: (0, 88, 3),
 307: (0, 116, 3),
 527: (61, 96, 3),
 541: (26, 3, 3),
 125: (106, 31, 3),
 331: (1, 37, 3),
 30: (1, 68, 2),
 76: (54, 41, 2),
 861: (89, 12, 3),
 298: (54, 25, 3),
 700: (73, 127, 3),
 942: (67, 24, 3),
 874: (96, 14, 3),
 407: (86, 46, 3),
 558: (122, 52, 3),
 752: (2, 64, 3),
 196: (86, 113, 3),
 274: (2, 78, 3),
 240: (2, 82, 3),
 345: (47, 35, 3),
 105: (76, 53, 3),
 164: (45, 69, 3),
 466: (2, 137, 3),
 593: (3, 23, 3),
 516: (69, 83, 3),
 74: (104, 70, 2),
 377: (3, 91, 3),
 157: (108, 46, 3),
 128: (93, 127, 3),
 175: (3, 129, 3),
 314: (11, 68, 3),
 750: (4, 8, 3),
 497: (72, 65, 3),
 258: (4, 47, 3),
 549: (27, 21, 3),
 70: (12, 96, 2),
 745: (4, 73, 3),
 289: (130, 33, 3),
 418: (4, 85, 3),
 351: (4, 105, 3),
 839: (4, 133, 3),
 786: (5, 18, 3),
 283: (115, 39, 3),
 366: (27, 100, 3),
 899: (54, 93, 3),
 652: (127, 53, 3),
 219: (5, 111, 3),
 468: (120, 123, 3),
 249: (6, 14, 3),

In [ ]:
    
print(f"There are {len(not_valid)} numbers that are not valid part numbers.")
print(f"However, ")
print(f"There are {len(valid_numbers)} part numbers that ARE valid!")
_test = str(valid_numbers[:10])[:-1] + " ... " + str(valid_numbers[-10:])[1:]
print(_test)
print(f"The sum of all valid part numbers is {final}.")

In [19]:
h = set((1, 2))
print(h[0])

TypeError: 'set' object is not subscriptable

In [38]:

print(f"There are {len(not_valid)} numbers that are not valid part numbers.")
print(f"However, ")
print(f"There are {len(valid_numbers)} part numbers that ARE valid!")
_test = str(valid_numbers[:10])[:-1] + " ... " + str(valid_numbers[-10:])[1:]
print(_test)
print(f"The sum of all valid part numbers is {final}.")

There are 134 numbers that are not valid part numbers.
However, 
There are 1073 part numbers that ARE valid!
[65, 998, 453, 845, 773, 307, 527, 125, 331, 76 ... 1, 292, 602, 321, 728, 3, 44, 291, 873, 461]
The sum of all valid part numbers is 530495.


There are 1073 part numbers that are valid.
There are 134 numbers that are not valid part numbers.


In [ ]:
not_valid

In [ ]:
x = [x for x in range(0, 50)]
y = [y for y in range(50, 0, -1)]
m = map(lambda a, b: a * b, x, y)
for x in list(m):
    print(x)

In [43]:
list_all = list(all_numbers)
list_valid = list(valid_numbers)
for i in list_all:
    if i not in list_valid:
        print(i)
        

In [47]:
list_all

[1,
 1,
 1,
 2,
 2,
 3,
 3,
 5,
 7,
 7,
 8,
 9,
 9,
 10,
 10,
 10,
 13,
 14,
 14,
 15,
 15,
 15,
 16,
 16,
 16,
 17,
 17,
 19,
 19,
 19,
 20,
 20,
 21,
 21,
 22,
 22,
 22,
 23,
 23,
 23,
 24,
 24,
 24,
 24,
 25,
 25,
 25,
 26,
 27,
 28,
 28,
 29,
 30,
 33,
 33,
 34,
 36,
 36,
 36,
 37,
 40,
 40,
 40,
 41,
 41,
 42,
 42,
 43,
 44,
 44,
 45,
 46,
 47,
 48,
 49,
 49,
 50,
 51,
 51,
 51,
 52,
 52,
 52,
 55,
 55,
 56,
 57,
 58,
 58,
 59,
 59,
 60,
 61,
 62,
 64,
 64,
 65,
 66,
 68,
 68,
 69,
 70,
 70,
 71,
 72,
 72,
 73,
 73,
 74,
 74,
 75,
 76,
 76,
 78,
 80,
 83,
 87,
 87,
 88,
 88,
 88,
 89,
 89,
 89,
 92,
 92,
 93,
 93,
 94,
 94,
 95,
 96,
 98,
 99,
 101,
 101,
 102,
 103,
 103,
 105,
 105,
 105,
 105,
 108,
 108,
 108,
 110,
 111,
 112,
 112,
 112,
 115,
 115,
 120,
 120,
 120,
 123,
 125,
 125,
 125,
 125,
 125,
 126,
 127,
 128,
 128,
 128,
 129,
 130,
 130,
 131,
 131,
 132,
 132,
 133,
 135,
 135,
 137,
 139,
 140,
 140,
 144,
 144,
 146,
 147,
 148,
 148,
 149,
 149,
 150,
 152,
 